In [27]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

In [28]:
# Initialize a network
# each node is implemented using dictionary
# each layer is a list of nodes, and network is a list of layers
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

In [29]:
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
	print(layer)


[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [30]:
# Calculate neuron activation for an input
# basically weighted sum of inputs to the neuron
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation


In [31]:
# Transfer neuron activation
# sigmoid function
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))


In [32]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = [] # store input for next layer/ output of this layer

        # traverse through each node and calculate its output
		for neuron in layer: # each neuron is a dictionary
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation) # storing output in the dictionary/neuron
			new_inputs.append(neuron['output'])

		inputs = new_inputs

	return inputs

In [33]:
# test forward propagation
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None] # None represents the bias input, it can be deleted 
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [34]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)


In [35]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
            # for hidden layers
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
            # for output layer
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [36]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


In [37]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1] # excluding the target value
		if i != 0:
            # for layers other than first hidden layer take input from previous layer
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
                # update each weight for the neuron
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            # updating the bias weight
			neuron['weights'][-1] += l_rate * neuron['delta']

In [38]:
# Train a network for a fixed number of epochs
# epoch refers to one full iteration of the dataset
# l_rate is the learning rate
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row) # get output using current weights
            
            # since the target/class value is known, probability for that value is 1 and rest is 0
			expected = [0 for i in range(n_outputs)] # initialize to zero
			expected[row[-1]] = 1 # set the class value to 1
            
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
        
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [39]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.0026279652850863837}]
[{'weights

In [40]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [41]:
# Test making predictions with the network
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
network = [[{'weights': [-1.482313569067226, 1.8308790073202204, 1.078381922048799]}, {'weights': [0.23244990332399884, 0.3621998343835864, 0.40289821191094327]}],
	[{'weights': [2.5001872433501404, 0.7887233511355132, -1.1026649757805829]}, {'weights': [-2.429350576245497, 0.8357651039198697, 1.0699217181280656]}]]
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


In [42]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [43]:
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

In [44]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [45]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [46]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [47]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [50]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [53]:
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'Crop_recommendation.csv'
dataset = load_csv(filename)
dataset.pop(0)

target_set = set(row[-1] for row in dataset)
target_index = {}
i = 0
for target in target_set :
    target_index[target] = i
    i+=1
    
print(target_index)

# replace class labels (strings) with integer equivalent (index)
for row in dataset:
    for i in range(len(row)-1):
        row[i] = float(row[i])
    row[-1] = int(target_index[row[-1]])

# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

{'coconut': 0, 'papaya': 1, 'banana': 2, 'orange': 3, 'rice': 4, 'maize': 5, 'apple': 6, 'kidneybeans': 7, 'grapes': 8, 'coffee': 9, 'mothbeans': 10, 'watermelon': 11, 'cotton': 12, 'muskmelon': 13, 'blackgram': 14, 'pomegranate': 15, 'chickpea': 16, 'pigeonpeas': 17, 'mungbean': 18, 'mango': 19, 'lentil': 20, 'jute': 21}
>epoch=0, lrate=0.300, error=1956.691
>epoch=1, lrate=0.300, error=1681.844
>epoch=2, lrate=0.300, error=1680.967
>epoch=3, lrate=0.300, error=1679.688
>epoch=4, lrate=0.300, error=1677.686
>epoch=5, lrate=0.300, error=1674.367
>epoch=6, lrate=0.300, error=1668.290
>epoch=7, lrate=0.300, error=1650.254
>epoch=8, lrate=0.300, error=1606.428
>epoch=9, lrate=0.300, error=1570.770
>epoch=10, lrate=0.300, error=1537.418
>epoch=11, lrate=0.300, error=1506.980
>epoch=12, lrate=0.300, error=1477.778
>epoch=13, lrate=0.300, error=1449.136
>epoch=14, lrate=0.300, error=1421.778
>epoch=15, lrate=0.300, error=1392.758
>epoch=16, lrate=0.300, error=1357.867
>epoch=17, lrate=0.300,

>epoch=204, lrate=0.300, error=287.130
>epoch=205, lrate=0.300, error=286.711
>epoch=206, lrate=0.300, error=286.291
>epoch=207, lrate=0.300, error=285.871
>epoch=208, lrate=0.300, error=285.451
>epoch=209, lrate=0.300, error=285.031
>epoch=210, lrate=0.300, error=284.610
>epoch=211, lrate=0.300, error=284.188
>epoch=212, lrate=0.300, error=283.766
>epoch=213, lrate=0.300, error=283.342
>epoch=214, lrate=0.300, error=282.917
>epoch=215, lrate=0.300, error=282.491
>epoch=216, lrate=0.300, error=282.063
>epoch=217, lrate=0.300, error=281.634
>epoch=218, lrate=0.300, error=281.203
>epoch=219, lrate=0.300, error=280.770
>epoch=220, lrate=0.300, error=280.335
>epoch=221, lrate=0.300, error=279.899
>epoch=222, lrate=0.300, error=279.460
>epoch=223, lrate=0.300, error=279.020
>epoch=224, lrate=0.300, error=278.578
>epoch=225, lrate=0.300, error=278.134
>epoch=226, lrate=0.300, error=277.689
>epoch=227, lrate=0.300, error=277.242
>epoch=228, lrate=0.300, error=276.794
>epoch=229, lrate=0.300, 

>epoch=415, lrate=0.300, error=232.479
>epoch=416, lrate=0.300, error=232.370
>epoch=417, lrate=0.300, error=232.261
>epoch=418, lrate=0.300, error=232.153
>epoch=419, lrate=0.300, error=232.045
>epoch=420, lrate=0.300, error=231.938
>epoch=421, lrate=0.300, error=231.832
>epoch=422, lrate=0.300, error=231.725
>epoch=423, lrate=0.300, error=231.620
>epoch=424, lrate=0.300, error=231.515
>epoch=425, lrate=0.300, error=231.410
>epoch=426, lrate=0.300, error=231.306
>epoch=427, lrate=0.300, error=231.203
>epoch=428, lrate=0.300, error=231.099
>epoch=429, lrate=0.300, error=230.997
>epoch=430, lrate=0.300, error=230.895
>epoch=431, lrate=0.300, error=230.793
>epoch=432, lrate=0.300, error=230.692
>epoch=433, lrate=0.300, error=230.591
>epoch=434, lrate=0.300, error=230.491
>epoch=435, lrate=0.300, error=230.391
>epoch=436, lrate=0.300, error=230.292
>epoch=437, lrate=0.300, error=230.193
>epoch=438, lrate=0.300, error=230.094
>epoch=439, lrate=0.300, error=229.997
>epoch=440, lrate=0.300, 

>epoch=128, lrate=0.300, error=351.247
>epoch=129, lrate=0.300, error=350.495
>epoch=130, lrate=0.300, error=349.754
>epoch=131, lrate=0.300, error=349.022
>epoch=132, lrate=0.300, error=348.299
>epoch=133, lrate=0.300, error=347.585
>epoch=134, lrate=0.300, error=346.878
>epoch=135, lrate=0.300, error=346.178
>epoch=136, lrate=0.300, error=345.485
>epoch=137, lrate=0.300, error=344.798
>epoch=138, lrate=0.300, error=344.117
>epoch=139, lrate=0.300, error=343.440
>epoch=140, lrate=0.300, error=342.768
>epoch=141, lrate=0.300, error=342.099
>epoch=142, lrate=0.300, error=341.434
>epoch=143, lrate=0.300, error=340.771
>epoch=144, lrate=0.300, error=340.111
>epoch=145, lrate=0.300, error=339.452
>epoch=146, lrate=0.300, error=338.794
>epoch=147, lrate=0.300, error=338.137
>epoch=148, lrate=0.300, error=337.480
>epoch=149, lrate=0.300, error=336.822
>epoch=150, lrate=0.300, error=336.164
>epoch=151, lrate=0.300, error=335.504
>epoch=152, lrate=0.300, error=334.843
>epoch=153, lrate=0.300, 

>epoch=339, lrate=0.300, error=266.176
>epoch=340, lrate=0.300, error=266.037
>epoch=341, lrate=0.300, error=265.899
>epoch=342, lrate=0.300, error=265.761
>epoch=343, lrate=0.300, error=265.625
>epoch=344, lrate=0.300, error=265.489
>epoch=345, lrate=0.300, error=265.354
>epoch=346, lrate=0.300, error=265.220
>epoch=347, lrate=0.300, error=265.087
>epoch=348, lrate=0.300, error=264.954
>epoch=349, lrate=0.300, error=264.822
>epoch=350, lrate=0.300, error=264.691
>epoch=351, lrate=0.300, error=264.561
>epoch=352, lrate=0.300, error=264.431
>epoch=353, lrate=0.300, error=264.302
>epoch=354, lrate=0.300, error=264.174
>epoch=355, lrate=0.300, error=264.046
>epoch=356, lrate=0.300, error=263.920
>epoch=357, lrate=0.300, error=263.793
>epoch=358, lrate=0.300, error=263.668
>epoch=359, lrate=0.300, error=263.543
>epoch=360, lrate=0.300, error=263.419
>epoch=361, lrate=0.300, error=263.295
>epoch=362, lrate=0.300, error=263.172
>epoch=363, lrate=0.300, error=263.050
>epoch=364, lrate=0.300, 

...

>epoch=473, lrate=0.300, error=430.443
>epoch=474, lrate=0.300, error=430.346
>epoch=475, lrate=0.300, error=430.250
>epoch=476, lrate=0.300, error=430.153
>epoch=477, lrate=0.300, error=430.057
>epoch=478, lrate=0.300, error=429.961
>epoch=479, lrate=0.300, error=429.866
>epoch=480, lrate=0.300, error=429.771
>epoch=481, lrate=0.300, error=429.676
>epoch=482, lrate=0.300, error=429.581
>epoch=483, lrate=0.300, error=429.487
>epoch=484, lrate=0.300, error=429.393
>epoch=485, lrate=0.300, error=429.300
>epoch=486, lrate=0.300, error=429.206
>epoch=487, lrate=0.300, error=429.113
>epoch=488, lrate=0.300, error=429.020
>epoch=489, lrate=0.300, error=428.928
>epoch=490, lrate=0.300, error=428.835
>epoch=491, lrate=0.300, error=428.743
>epoch=492, lrate=0.300, error=428.652
>epoch=493, lrate=0.300, error=428.560
>epoch=494, lrate=0.300, error=428.469
>epoch=495, lrate=0.300, error=428.378
>epoch=496, lrate=0.300, error=428.288
>epoch=497, lrate=0.300, error=428.197
>epoch=498, lrate=0.300, 

>epoch=186, lrate=0.300, error=381.948
>epoch=187, lrate=0.300, error=380.882
>epoch=188, lrate=0.300, error=379.829
>epoch=189, lrate=0.300, error=378.790
>epoch=190, lrate=0.300, error=377.763
>epoch=191, lrate=0.300, error=376.748
>epoch=192, lrate=0.300, error=375.746
>epoch=193, lrate=0.300, error=374.756
>epoch=194, lrate=0.300, error=373.778
>epoch=195, lrate=0.300, error=372.812
>epoch=196, lrate=0.300, error=371.858
>epoch=197, lrate=0.300, error=370.914
>epoch=198, lrate=0.300, error=369.981
>epoch=199, lrate=0.300, error=369.060
>epoch=200, lrate=0.300, error=368.148
>epoch=201, lrate=0.300, error=367.247
>epoch=202, lrate=0.300, error=366.356
>epoch=203, lrate=0.300, error=365.474
>epoch=204, lrate=0.300, error=364.602
>epoch=205, lrate=0.300, error=363.739
>epoch=206, lrate=0.300, error=362.885
>epoch=207, lrate=0.300, error=362.039
>epoch=208, lrate=0.300, error=361.201
>epoch=209, lrate=0.300, error=360.372
>epoch=210, lrate=0.300, error=359.550
>epoch=211, lrate=0.300, 

>epoch=397, lrate=0.300, error=283.475
>epoch=398, lrate=0.300, error=283.326
>epoch=399, lrate=0.300, error=283.179
>epoch=400, lrate=0.300, error=283.033
>epoch=401, lrate=0.300, error=282.887
>epoch=402, lrate=0.300, error=282.742
>epoch=403, lrate=0.300, error=282.598
>epoch=404, lrate=0.300, error=282.454
>epoch=405, lrate=0.300, error=282.311
>epoch=406, lrate=0.300, error=282.169
>epoch=407, lrate=0.300, error=282.028
>epoch=408, lrate=0.300, error=281.887
>epoch=409, lrate=0.300, error=281.747
>epoch=410, lrate=0.300, error=281.608
>epoch=411, lrate=0.300, error=281.469
>epoch=412, lrate=0.300, error=281.331
>epoch=413, lrate=0.300, error=281.194
>epoch=414, lrate=0.300, error=281.058
>epoch=415, lrate=0.300, error=280.922
>epoch=416, lrate=0.300, error=280.786
>epoch=417, lrate=0.300, error=280.652
>epoch=418, lrate=0.300, error=280.517
>epoch=419, lrate=0.300, error=280.384
>epoch=420, lrate=0.300, error=280.251
>epoch=421, lrate=0.300, error=280.119
>epoch=422, lrate=0.300, 

>epoch=110, lrate=0.300, error=339.556
>epoch=111, lrate=0.300, error=337.862
>epoch=112, lrate=0.300, error=336.207
>epoch=113, lrate=0.300, error=334.588
>epoch=114, lrate=0.300, error=333.003
>epoch=115, lrate=0.300, error=331.449
>epoch=116, lrate=0.300, error=329.924
>epoch=117, lrate=0.300, error=328.425
>epoch=118, lrate=0.300, error=326.950
>epoch=119, lrate=0.300, error=325.496
>epoch=120, lrate=0.300, error=324.060
>epoch=121, lrate=0.300, error=322.641
>epoch=122, lrate=0.300, error=321.236
>epoch=123, lrate=0.300, error=319.841
>epoch=124, lrate=0.300, error=318.456
>epoch=125, lrate=0.300, error=317.077
>epoch=126, lrate=0.300, error=315.702
>epoch=127, lrate=0.300, error=314.328
>epoch=128, lrate=0.300, error=312.955
>epoch=129, lrate=0.300, error=311.579
>epoch=130, lrate=0.300, error=310.201
>epoch=131, lrate=0.300, error=308.818
>epoch=132, lrate=0.300, error=307.432
>epoch=133, lrate=0.300, error=306.043
>epoch=134, lrate=0.300, error=304.653
>epoch=135, lrate=0.300, 

>epoch=321, lrate=0.300, error=214.694
>epoch=322, lrate=0.300, error=214.565
>epoch=323, lrate=0.300, error=214.438
>epoch=324, lrate=0.300, error=214.311
>epoch=325, lrate=0.300, error=214.185
>epoch=326, lrate=0.300, error=214.060
>epoch=327, lrate=0.300, error=213.936
>epoch=328, lrate=0.300, error=213.813
>epoch=329, lrate=0.300, error=213.690
>epoch=330, lrate=0.300, error=213.569
>epoch=331, lrate=0.300, error=213.448
>epoch=332, lrate=0.300, error=213.328
>epoch=333, lrate=0.300, error=213.209
>epoch=334, lrate=0.300, error=213.091
>epoch=335, lrate=0.300, error=212.973
>epoch=336, lrate=0.300, error=212.856
>epoch=337, lrate=0.300, error=212.741
>epoch=338, lrate=0.300, error=212.625
>epoch=339, lrate=0.300, error=212.511
>epoch=340, lrate=0.300, error=212.397
>epoch=341, lrate=0.300, error=212.284
>epoch=342, lrate=0.300, error=212.172
>epoch=343, lrate=0.300, error=212.061
>epoch=344, lrate=0.300, error=211.950
>epoch=345, lrate=0.300, error=211.840
>epoch=346, lrate=0.300, 